##✅ Streamlit Code for FinBot:
- Load fine-tuned model using transformers + bitsandbytes

- Build a Streamlit interface

- Set up a function to generate answers

- Run the chatbot in a browser

In [ ]:
%%writefile app.py

import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

# App title and instructions
st.set_page_config(page_title="FinBot - Financial Literacy Chatbot")
st.title("💰 FinBot - Your Financial Literacy Assistant")
st.markdown("Ask me anything about budgeting, saving, investing, or credit!")

# Load model and tokenizer
@st.cache_resource
def load_model():
    model_id = "0xroyce/Plutus-3B"
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        load_in_8bit=True,
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    return tokenizer, model

tokenizer, model = load_model()

# Function to generate response
def get_response(question):
    input_ids = tokenizer.encode(question, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split(question)[-1].strip()

# Chat input and history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_input = st.text_input("💬 Enter your financial question here:")

if user_input:
    with st.spinner("Thinking..."):
        answer = get_response(user_input)
        st.session_state.chat_history.append((user_input, answer))

# Show chat history
for user_msg, bot_msg in reversed(st.session_state.chat_history):
    st.markdown(f"**You:** {user_msg}")
    st.markdown(f"**FinBot:** {bot_msg}")
    st.markdown("---")
